# Feature engineering 
Binning the categorical features

Import the packages we need

In [ ]:
import itertools
from functions import *

# Data Science
import pandas as pd
import numpy as np

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Load and transform the data
First we load and show the data in pandas dataframe. Then we can start transforming the columns.

In [ ]:
df_train = pd.read_excel('D:/krk-datascientist-master/case2/use_case_data.xlsx')
df_train.head(5)

Transform some of the features

In [ ]:
#Merge REGION and MARKET into one feature
df_train['REGION_MARKET']=df_train['REGION'].astype(str)+'_'+df_train['MARKET'].astype(str)

#Encode Date into Months
df_train['LAUNCHMONTH']=df_train['NPLLAUNCHDATE'].astype(str).str[-2:]

#Add Target variable
df_train['Successful']=np.where(df_train['Market_Share']<0.007, 0, 1)

#SPECIALFLAVOR - we fill the missing values with a 'no flavor' category under the assumption that the product did not have a flavor 
df_train['SPECIALFLAVOR']=df_train['SPECIALFLAVOR'].fillna('NOFLAVOR')

#TIPCOLOR - we fill the missing values with a 'no color' category under the assumption that the product did not have a color
df_train['TIPCOLOR']=df_train['TIPCOLOR'].fillna('NOCOLOR')

# Drop the features we do not need
Some of the features we do not need, since they does not have any meaning in the model.

In [ ]:
# remove columns with no information and missing values (more than 50%)

# BRANDSUBFAMILYGROUPING beacuse is the same as BRANDSUBFAMILY
# BRMID is an ID
# ISREPLACEMENT has always the same value
# ITEMSHAPE is either NaN or 1
# LATESTPERIODINDEX has always the same value

# REGION and MARKET are REGION_MARKET
# RTYPE has many missing values

# NPLLAUNCHDATE is represented using only months
# NPLLAUNCHYEAR not is needed
# Market_Share is the Successful variable

df_train=df_train.drop(['BRANDSUBFAMILYGROUPING','BRMID','ISREPLACEMENT','ITEMSHAPE', 'LATESTPERIODINDEX', 'RTYPE',
                         'REGION', 'MARKET','NPLLAUNCHDATE','NPLLAUNCHYEAR','Market_Share'],axis=1)

# Show the new data frame
df_train.head(5)

# Binning on some of the categorical features

In [ ]:
def column_frequency(df, column):
    """This is a function that counts the number of category repretitions in a given feature.
        It returns a dataframe.
        
        :param dataframe df             : the dataframe for grouping
        :param object column            : the column that we want to count values on
    """
    
    df = df.astype(str).groupby(column, as_index=False).size()
    df = df.reset_index()
    df = df.rename(columns={0: 'Counts'})
    df = df.sort_values('Counts', ascending=False)
    return df

In [ ]:
def plot_merged_category(df, column):
     """This is a function that plots the number of category repretitions in a given feature.
        It returns a dataframe.
        
        :param dataframe df             : the dataframe for grouping
        :param object column            : the column that we want to plot
    """
    df_merged=column_frequency(df,[column,'Successful'])
    df_merged = df_merged.rename(columns={'Counts':'Number of category repetitions'})
    plt.figure(1, figsize=(18, 10))
    plt.ylabel('',fontsize=20)
    plt.xlabel('',fontsize=20)
    plt.xticks(rotation=90, fontsize=8)
    plt.title(column+' '+'category frequencies after merging', fontsize=20)
    sns.barplot(x=df_merged[column], y=df_merged['Number of category repetitions'], hue=df_merged['Successful'])

In [ ]:
df_train.columns

In [ ]:
#BRM
brm=column_frequency(df_train,'BRM')

#Each BRM category that has 1 or 2 appearances in the dataset is merged into new category
brm['BRM_MERGED']=np.where(brm['Counts']<3,'BRM_1_2',brm['BRM'])
#Each BRM category that has 3 or 4 appearances in the dataset is merged into new category
brm['BRM_MERGED']=np.where((brm['Counts']>=3)&(brm['Counts']<5),'BRM_3_4',brm['BRM_MERGED'])
#Each BRM category that has 5 or 6 appearances in the dataset is merged into new category
brm['BRM_MERGED']=np.where((brm['Counts']>=5)&(brm['Counts']<7),'BRM_5_6',brm['BRM_MERGED'])
#Each BRM category that has 7,8 or 9 appearances in the dataset is merged into new category
brm['BRM_MERGED']=np.where((brm['Counts']>=7)&(brm['Counts']<10),'BRM_7_8_9',brm['BRM_MERGED'])
#the rest goes to another category
brm['BRM_MERGED']=np.where((brm['Counts']>=10),'BRM_10',brm['BRM_MERGED'])

brm=brm[['BRM','BRM_MERGED']]
df_train=pd.merge(df_train,brm,on='BRM')
plot_merged_category(df_train,'BRM_MERGED')

In [ ]:
#BRANDONMARKET
brand_on_market=column_frequency(df_train, 'BRANDONMARKET')
#Each BRANDONMARKET category that  has 1 appearance in the dataset is merged into new category
brand_on_market['BRANDONMARKET_MERGED']=np.where(brand_on_market['Counts']==1,'BRANDONMARKET_1',brand_on_market['BRANDONMARKET'])
#Each BRANDONMARKET category that  has 2 appearances in the dataset is merged into new category
brand_on_market['BRANDONMARKET_MERGED']=np.where((brand_on_market['Counts']==2),'BRANDONMARKET_2'
                                                 ,brand_on_market['BRANDONMARKET_MERGED'])
#Each BRANDONMARKET category that  has 3 or 4 appearances in the dataset is merged into new category
brand_on_market['BRANDONMARKET_MERGED']=np.where((brand_on_market['Counts']>=3)&(brand_on_market['Counts']<5),'BRANDONMARKET_3_4',
                                       brand_on_market['BRANDONMARKET_MERGED'])
#Each BRANDONMARKET category that  has 5 or 6 appearances in the dataset is merged into new category
brand_on_market['BRANDONMARKET_MERGED']=np.where(((brand_on_market['Counts']>=5)&(brand_on_market['Counts']<7)),'BRANDONMARKET_5_6',
                                                    brand_on_market['BRANDONMARKET_MERGED'])
#Each BRANDONMARKET category that  has 7,8 or 9 appearances in the dataset is merged into new category
brand_on_market['BRANDONMARKET_MERGED']=np.where(((brand_on_market['Counts']>=7)&(brand_on_market['Counts']<10)),'BRANDONMARKET_7_8_9',
                                                    brand_on_market['BRANDONMARKET_MERGED'])
#the rest goes to another category
brand_on_market['BRANDONMARKET_MERGED']=np.where((brand_on_market['Counts']>=10),'BRANDONMARKET_10',brand_on_market['BRANDONMARKET_MERGED'])

brand_on_market=brand_on_market[['BRANDONMARKET','BRANDONMARKET_MERGED']]
df_train=pd.merge(df_train,brand_on_market,on='BRANDONMARKET')
df_train[['BRANDONMARKET','BRANDONMARKET_MERGED']].head()
df_train['BRANDONMARKET_MERGED'].value_counts()
plot_merged_category(df_train, 'BRANDONMARKET_MERGED')

In [ ]:
#MARKETEDBRAND

marketed_brand=column_frequency(df_train, 'MARKETEDBRAND')
#Each MARKETEDBRAND category that  has 1 appearance in the dataset is merged into new category
marketed_brand['MARKETEDBRAND_MERGED']=np.where(marketed_brand['Counts']==1,'MARKETEDBRAND_1',marketed_brand['MARKETEDBRAND'])
#Each MARKETEDBRAND category that  has 2 appearances in the dataset is merged into new category
marketed_brand['MARKETEDBRAND_MERGED']=np.where((marketed_brand['Counts']==2),'MARKETEDBRAND_2',
                                                marketed_brand['MARKETEDBRAND_MERGED'])
#Each MARKETEDBRAND category that  has 3 or 4 appearances in the dataset is merged into new category
marketed_brand['MARKETEDBRAND_MERGED']=np.where(((marketed_brand['Counts']>=3) & (marketed_brand['Counts']<5)),
                                                'MARKETEDBRAND_3_4',marketed_brand['MARKETEDBRAND_MERGED'])
#Each MARKETEDBRAND category that  has 5 or 6 appearances in the dataset is merged into new category
marketed_brand['MARKETEDBRAND_MERGED']=np.where(((marketed_brand['Counts']>=5)&(marketed_brand['Counts']<7)),'MARKETEDBRAND_5_6',marketed_brand['MARKETEDBRAND_MERGED'])
#Each MARKETEDBRAND category that  has 7,8 or 9 appearances in the dataset is merged into new category
marketed_brand['MARKETEDBRAND_MERGED']=np.where(((marketed_brand['Counts']>=7)&(marketed_brand['Counts']<10)),'MARKETEDBRAND_7_8_9',marketed_brand['MARKETEDBRAND_MERGED'])

#the rest goes to another category
marketed_brand['MARKETEDBRAND_MERGED']=np.where((marketed_brand['Counts']>=10),'MARKETEDBRAND_10',
                                                marketed_brand['MARKETEDBRAND_MERGED'])

marketed_brand=marketed_brand[['MARKETEDBRAND','MARKETEDBRAND_MERGED']]
df_train=pd.merge(df_train,marketed_brand,on='MARKETEDBRAND')
df_train[['MARKETEDBRAND','MARKETEDBRAND_MERGED']].head()
df_train['MARKETEDBRAND_MERGED'].value_counts()
plot_merged_category(df_train,'MARKETEDBRAND_MERGED')

In [ ]:
#BRANDDIFFERENTIATOR
brand_dif=column_frequency(df_train,'BRANDDIFFERENTIATOR')
#Each BRANDDIFFERENTIATOR category that  has 1 appearance in the dataset is merged into new category
brand_dif['BRANDDIFFERENTIATOR_MERGED']=np.where(brand_dif['Counts']==1,'BRANDDIFFERENTIATOR_1',brand_dif['BRANDDIFFERENTIATOR'])
#Each BRANDDIFFERENTIATOR category that  has 2 appearances in the dataset is merged into new category
brand_dif['BRANDDIFFERENTIATOR_MERGED']=np.where((brand_dif['Counts']==2),'BRANDDIFFERENTIATOR_2',
                                                brand_dif['BRANDDIFFERENTIATOR_MERGED'])
#Each BRANDDIFFERENTIATOR category that  has 3 or 4 appearances in the dataset is merged into new category
brand_dif['BRANDDIFFERENTIATOR_MERGED']=np.where(((brand_dif['Counts']>=3) & (brand_dif['Counts']<5)),
                                                'BRANDDIFFERENTIATOR_3_4',brand_dif['BRANDDIFFERENTIATOR_MERGED'])
#Each BRANDDIFFERENTIATOR category that  has 5 or 6 appearances in the dataset is merged into new category
brand_dif['BRANDDIFFERENTIATOR_MERGED']=np.where(((brand_dif['Counts']>=5)& (brand_dif['Counts']<7)),'BRANDDIFFERENTIATOR_5_6',brand_dif['BRANDDIFFERENTIATOR_MERGED'])
#Each BRANDDIFFERENTIATOR category that  has 7,8 or 9 appearances in the dataset is merged into new category
brand_dif['BRANDDIFFERENTIATOR_MERGED']=np.where(((brand_dif['Counts']>=7) &(brand_dif['Counts']<10)),'BRANDDIFFERENTIATOR_7_8_9',brand_dif['BRANDDIFFERENTIATOR_MERGED'])

#the rest goes to another category
brand_dif['BRANDDIFFERENTIATOR_MERGED']=np.where((brand_dif['Counts']>=10),'BRANDDIFFERENTIATOR_10',
                                                brand_dif['BRANDDIFFERENTIATOR_MERGED'])

brand_dif=brand_dif[['BRANDDIFFERENTIATOR','BRANDDIFFERENTIATOR_MERGED']]
df_train=pd.merge(df_train,brand_dif,on='BRANDDIFFERENTIATOR')
df_train[['BRANDDIFFERENTIATOR','BRANDDIFFERENTIATOR_MERGED']].head()
df_train['BRANDDIFFERENTIATOR_MERGED'].value_counts()
plot_merged_category(df_train,'BRANDDIFFERENTIATOR_MERGED')

BRANDDIFFERENTIATOR 245

In [ ]:
#ITEMSCODE
item_code = column_frequency(df_train, 'ITEMSCODE')
#Each ITEMSCODE category that  has 1 appearance in the dataset is merged into new category
item_code['ITEMSCODE_MERGED']=np.where(item_code['Counts']==1,'ITEMSCODE_1',item_code['ITEMSCODE'])
##Each ITEMSCODE category that  has 2 or 3 appearances in the dataset is merged into new category
item_code['ITEMSCODE_MERGED']=np.where(((item_code['Counts']>=2) & (item_code['Counts']<4)),'ITEMSCODE_2_3',item_code['ITEMSCODE_MERGED'])

#the rest goes to another category
item_code['ITEMSCODE_MERGED']=np.where((item_code['Counts']>=4),'ITEMSCODE_4',
                                                item_code['ITEMSCODE_MERGED'])

item_code['ITEMSCODE']=item_code['ITEMSCODE'].astype('int64')
item_code=item_code[['ITEMSCODE','ITEMSCODE_MERGED']]
df_train=pd.merge(df_train,item_code,on='ITEMSCODE')
# Transform the ITEMSCODE_MERGED to categorical
df_train['ITEMSCODE_MERGED']=df_train['ITEMSCODE_MERGED'].astype('category')

df_train['ITEMSCODE_MERGED'].value_counts()
plot_merged_category(df_train, 'ITEMSCODE_MERGED')

In [ ]:
#BLDIMAGE
image = column_frequency(df_train,'BLDIMAGE')
#Each BLDIMAGE category that  has from 1 to 7 appearances in the dataset is merged into new category
image['BLDIMAGE_MERGED']=np.where((image['Counts']>=1)&(image['Counts']<10),'BLDIMAGE_1_3_5_6_7',image['BLDIMAGE'])
#the rest goes to another category
image['BLDIMAGE_MERGED']=np.where((image['Counts']>=10),'BLDIMAGE_10',
                                                image['BLDIMAGE_MERGED'])

image=image[['BLDIMAGE','BLDIMAGE_MERGED']]
df_train=pd.merge(df_train,image,on='BLDIMAGE')
df_train['BLDIMAGE_MERGED'].value_counts()
plot_merged_category(df_train, 'BLDIMAGE_MERGED')

In [ ]:
#PRICECLASS
priceclass=column_frequency(df_train, 'PRICECLASS')
#Each PRICECLASS category that  has from 1 to 9 appearances in the dataset is merged into new category
priceclass['PRICECLASS_MERGED']=np.where((priceclass['Counts']<10),'PRICECLASS_1_9',priceclass['PRICECLASS'])
#the rest goes to another category
priceclass['PRICECLASS_MERGED']=np.where((priceclass['Counts']>=10),'PRICECLASS_10',
                                                priceclass['PRICECLASS_MERGED'])

priceclass=priceclass[['PRICECLASS','PRICECLASS_MERGED']]
df_train=pd.merge(df_train,priceclass,on='PRICECLASS')
df_train['PRICECLASS_MERGED'].value_counts()
plot_merged_category(df_train,'PRICECLASS_MERGED')

In [ ]:
#LOCALCLASS
localclass = column_frequency(df_train, 'LOCALCLASS')
#Each LOCALCLASS category that  has from 1 to 9 appearances in the dataset is merged into new category
localclass['LOCALCLASS_MERGED']=np.where((localclass['Counts']<10),'LOCALCLASS_1_9',localclass['LOCALCLASS'])
#the rest goes to another category
localclass['LOCALCLASS_MERGED']=np.where((localclass['Counts']>=10),'LOCALCLASS_10',
                                                localclass['LOCALCLASS_MERGED'])


localclass=localclass[['LOCALCLASS','LOCALCLASS_MERGED']]
df_train=pd.merge(df_train,localclass,on='LOCALCLASS')
df_train['LOCALCLASS_MERGED'].value_counts()
plot_merged_category(df_train, 'LOCALCLASS_MERGED')

In [ ]:
#PCKT
pckt = column_frequency(df_train, 'PCKT')
#Each PCKT category that  has from 1 to 3 appearances in the dataset is merged into new category
pckt['PCKT_MERGED']=np.where((pckt['Counts']<4),'PCKT_1_2_3',pckt['PCKT'])
#Each PCKT category that  has from 4 to 9 appearances in the dataset is merged into new category
pckt['PCKT_MERGED']=np.where(((pckt['Counts']>=4) & (pckt['Counts']<10)),
                                           'PCKT_4_9',pckt['PCKT_MERGED'])
#the rest goes to another category
pckt['PCKT_MERGED']=np.where((pckt['Counts']>=10),'PCKT_10',
                                                pckt['PCKT_MERGED'])
pckt=pckt[['PCKT','PCKT_MERGED']]
df_train=pd.merge(df_train,pckt,on='PCKT')
df_train['PCKT_MERGED'].value_counts()
plot_merged_category(df_train, 'PCKT_MERGED')

PCKT24

In [ ]:
#SPECIALFLAVOR
df_train['SPECIALFLAVOR']=df_train['SPECIALFLAVOR'].fillna('NOSPECIALFLAVOR')
specialflavor = column_frequency(df_train, 'SPECIALFLAVOR')
#Each SPECIALFLAVOR category that  has from 1 to 10 appearances in the dataset is merged into new category
specialflavor['SPECIALFLAVOR_MERGED']=np.where((specialflavor['Counts']<10),'SPECIALFLAVOR_1_10',specialflavor['SPECIALFLAVOR'])
#the rest goes to another category
specialflavor['SPECIALFLAVOR_MERGED']=np.where((specialflavor['Counts']>=10),'SPECIALFLAVOR_10',
                                                specialflavor['SPECIALFLAVOR_MERGED'])

specialflavor=specialflavor[['SPECIALFLAVOR','SPECIALFLAVOR_MERGED']]
df_train=pd.merge(df_train,specialflavor,on='SPECIALFLAVOR')
df_train['SPECIALFLAVOR_MERGED'].value_counts()
plot_merged_category(df_train,'SPECIALFLAVOR_MERGED' )

In [ ]:
#THICATEGORY
thi_category = column_frequency(df_train,'THICATEGORY')

#Each THICATEGORY category that  has from 1 to 100 appearances in the dataset is merged into new category
thi_category['THICATEGORY_MERGED']=np.where((thi_category['Counts']<100),'THICATEGORY_1_99',thi_category['THICATEGORY'])
#the rest goes to another category
thi_category['THICATEGORY_MERGED']=np.where((thi_category['Counts']>=100),'THICATEGORY_100',
                                                thi_category['THICATEGORY_MERGED'])

thi_category=thi_category[['THICATEGORY','THICATEGORY_MERGED']]
df_train=pd.merge(df_train,thi_category,on='THICATEGORY')
df_train['THICATEGORY_MERGED'].value_counts()
plot_merged_category(df_train,'THICATEGORY_MERGED' )

In [ ]:
#BRANDFAMILY
brand_family = column_frequency(df_train, 'BRANDFAMILY')
#Each BRANDFAMILY category that  has  1 appearance in the dataset is merged into new category
brand_family['BRANDFAMILY_MERGED']=np.where((brand_family['Counts']==1),'BRANDFAMILY_1',brand_family['BRANDFAMILY'])
#Each BRANDFAMILY category that  has  2 or 3 appearances in the dataset is merged into new category
brand_family['BRANDFAMILY_MERGED']=np.where(((brand_family['Counts']>=2) & (brand_family['Counts']<4)),
                                          'BRANDFAMILY_2_3',brand_family['BRANDFAMILY_MERGED'])
#Each BRANDFAMILY category that  has  4,5 or 6 appearances in the dataset is merged into new category
brand_family['BRANDFAMILY_MERGED']=np.where(((brand_family['Counts']>=4) & (brand_family['Counts']<7)),
                                           'BRANDFAMILY_4_5_6',brand_family['BRANDFAMILY_MERGED'])
#Each BRANDFAMILY category that  has  7,8 or 9 appearances in the dataset is merged into new category
brand_family['BRANDFAMILY_MERGED']=np.where((brand_family['Counts']>=7)&(brand_family['Counts']<10),'BRANDFAMILY_7_8_9',brand_family['BRANDFAMILY_MERGED'])
#the rest goes to another category
brand_family['BRANDFAMILY_MERGED']=np.where((brand_family['Counts']>=10),'BRANDFAMILY_10',
                                                brand_family['BRANDFAMILY_MERGED'])

brand_family=brand_family[['BRANDFAMILY','BRANDFAMILY_MERGED']]
df_train=pd.merge(df_train,brand_family,on='BRANDFAMILY')
df_train['BRANDFAMILY_MERGED'].value_counts()
plot_merged_category(df_train, 'BRANDFAMILY_MERGED')

In [ ]:
#BRANDSUBFAMILY
brand_subfamily = column_frequency(df_train, 'BRANDSUBFAMILY')
#Each BRANDSUBFAMILY category that  has  1 appearance in the dataset is merged into new category
brand_subfamily['BRANDSUBFAMILY_MERGED']=np.where((brand_subfamily['Counts']==1),'BRANDSUBFAMILY_1',brand_subfamily['BRANDSUBFAMILY'])
#Each BRANDSUBFAMILY category that  has  2 or 3 appearances in the dataset is merged into new category
brand_subfamily['BRANDSUBFAMILY_MERGED']=np.where(((brand_subfamily['Counts']>=2) & (brand_subfamily['Counts']<4)),
                               'BRANDSUBFAMILY_2_3',brand_subfamily['BRANDSUBFAMILY_MERGED'])
#Each BRANDSUBFAMILY category that  has  4,5 or 6 appearances in the dataset is merged into new category
brand_subfamily['BRANDSUBFAMILY_MERGED']=np.where(((brand_subfamily['Counts']>=4) & (brand_subfamily['Counts']<7)),
                                         'BRANDSUBFAMILY_4_5_6',brand_subfamily['BRANDSUBFAMILY_MERGED'])
#Each BRANDSUBFAMILY category that  has  7,8 or 9 appearances in the dataset is merged into new category
brand_subfamily['BRANDSUBFAMILY_MERGED']=np.where((brand_subfamily['Counts']>=7)&(brand_subfamily['Counts']<10),'BRANDSUBFAMILY_7_8_9',
                                               brand_subfamily['BRANDSUBFAMILY_MERGED'])
#the rest goes to another category
brand_subfamily['BRANDSUBFAMILY_MERGED']=np.where((brand_subfamily['Counts']>=10),'BRANDSUBFAMILY_10',
                                                brand_subfamily['BRANDSUBFAMILY_MERGED'])

brand_subfamily=brand_subfamily[['BRANDSUBFAMILY','BRANDSUBFAMILY_MERGED']]
df_train=pd.merge(df_train,brand_subfamily,on='BRANDSUBFAMILY')
df_train['BRANDSUBFAMILY_MERGED'].value_counts()
plot_merged_category(df_train, 'BRANDSUBFAMILY_MERGED')

BRANDSUMFAMILY141 BRANDSUBFAMILY52 BRANDSUBFAMILY122

In [ ]:
#BRANDSUBFAMILYGROUP
brand_subfamily_gr = column_frequency(df_train, 'BRANDSUBFAMILYGROUP')
#Each BRANDSUBFAMILYGROUP category that  has  1 appearance in the dataset is merged into new category
brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED']=np.where((brand_subfamily_gr['Counts']==1),'BRANDSUBFAMILYGROUP_1',
                                                     brand_subfamily_gr['BRANDSUBFAMILYGROUP'])
#Each BRANDSUBFAMILYGROUP category that  has  2 or 3 appearances in the dataset is merged into new category
brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED']=np.where(((brand_subfamily_gr['Counts']>=2) & (brand_subfamily_gr['Counts']<4)),
                                           'BRANDSUBFAMILYGROUP_2_3',brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED'])
#Each BRANDSUBFAMILYGROUP category that  has  4,5 or 6 appearances in the dataset is merged into new category
brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED']=np.where(((brand_subfamily_gr['Counts']>=4) & (brand_subfamily_gr['Counts']<7)),
                                           'BRANDSUBFAMILYGROUP_4_5_6',brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED'])
#Each BRANDSUBFAMILYGROUP category that  has  7,8 or 9 appearances in the dataset is merged into new category
brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED']=np.where((brand_subfamily_gr['Counts']>=7)&(brand_subfamily_gr['Counts']<10),'BRANDSUBFAMILYGROUP_7_8_9',
                                               brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED'])
#the rest goes to another category
brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED']=np.where((brand_subfamily_gr['Counts']>=10),'BRANDSUBFAMILYGROUP_10',
                                                brand_subfamily_gr['BRANDSUBFAMILYGROUP_MERGED'])

brand_subfamily_gr=brand_subfamily_gr[['BRANDSUBFAMILYGROUP','BRANDSUBFAMILYGROUP_MERGED']]
df_train=pd.merge(df_train,brand_subfamily_gr,on='BRANDSUBFAMILYGROUP')
df_train['BRANDSUBFAMILYGROUP_MERGED'].value_counts()
plot_merged_category(df_train, 'BRANDSUBFAMILYGROUP_MERGED')

In [ ]:
#BRANDSUBFAMILYLINE
brand_subfamily_line = column_frequency(df_train,'BRANDSUBFAMILYLINE')
#Each BRANDSUBFAMILYLINE category that  has  1 appearance in the dataset is merged into new category
brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED']=np.where((brand_subfamily_line['Counts']==1),'BRANDSUBFAMILYLINE_1',
                                                     brand_subfamily_line['BRANDSUBFAMILYLINE'])
#Each BRANDSUBFAMILYLINE category that  has  2 or 3 appearances in the dataset is merged into new category
brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED']=np.where(((brand_subfamily_line['Counts']>=2) & (brand_subfamily_line['Counts']<4)),
                                           'BRANDSUBFAMILYLINE_2_3',brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED'])
#Each BRANDSUBFAMILYLINE category that  has  4,5 or 6 appearances in the dataset is merged into new category
brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED']=np.where(((brand_subfamily_line['Counts']>=4) & (brand_subfamily_line['Counts']<7)),
                                           'BRANDSUBFAMILYLINE_4_5_6',brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED'])
#Each BRANDSUBFAMILYLINE category that  has  7,8 or 9 appearances in the dataset is merged into new category
brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED']=np.where((brand_subfamily_line['Counts']>=7)&(brand_subfamily_line['Counts']<10),'BRANDSUBFAMILYLINE_7_8_9',
                                               brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED'])
#the rest goes to another category
brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED']=np.where((brand_subfamily_line['Counts']>=10),'BRANDSUBFAMILYLINE_10',
                                                brand_subfamily_line['BRANDSUBFAMILYLINE_MERGED'])

brand_subfamily_line=brand_subfamily_line[['BRANDSUBFAMILYLINE','BRANDSUBFAMILYLINE_MERGED']]
df_train=pd.merge(df_train,brand_subfamily_line,on='BRANDSUBFAMILYLINE')
df_train['BRANDSUBFAMILYLINE_MERGED'].value_counts()
plot_merged_category(df_train,'BRANDSUBFAMILYLINE_MERGED' )

Drop the columns for which there is a binned column

In [ ]:
df_train.shape

In [ ]:
#REGION_MARKET
#BRM
#BRANDONMARKET
#MARKETEDBRAND
#BRANDDIFFERENTIATOR
#ITEMSCODE
#BLDIMAGE
#PRICECLASS
#LOCALCLASS
#PCKT
#SPECIALFLAVOR
#THICATEGORY
#BRANDFAMILY
#BRANDSUBFAMILY
#BRANDSUBFAMILYGROUP
#BRANDSUBFAMILYLINE

df_train=df_train.drop(['BRM','BRANDONMARKET','MARKETEDBRAND','BRANDDIFFERENTIATOR','ITEMSCODE', 'BLDIMAGE', 
                        'PRICECLASS', 'LOCALCLASS', 'PCKT','SPECIALFLAVOR','THICATEGORY','BRANDFAMILY','BRANDSUBFAMILY',
                       'BRANDSUBFAMILYGROUP', 'BRANDSUBFAMILYLINE'],axis=1)

df_train.shape

Save the dataset into a new file

In [ ]:
writer = pd.ExcelWriter('D:/krk-datascientist-master/case2/after_feature_engineering.xlsx')
df_train.to_excel(writer,'Sheet1')
writer.save()